# Interpolating along a section
In this example we demonstrate how to create a gridded field from data on a section, for instance a set of profiles.

In [ ]:
using DIVAnd
using DelimitedFiles
using Makie, CairoMakie
using Interpolations
using Statistics
include("../config.jl")

## Files and directories

In [ ]:
sectionfile = joinpath(datadir, "temp_seasoar_section_5.dat")
sectionfile_URL = make_dox_url("UmvV8JsQt3eS6PO")
datafile = sectionfile
download_check(sectionfile, sectionfile_URL)

## Read and plot data 

In [ ]:
data = readdlm(datafile)
lat = data[:, 1]
depth = data[:, 2]
T = data[:, 3];
lon = -11.5 * ones(length(lat));

### Plot temperature section

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], xlabel = "Latitude (°N)", ylabel = "Depth (m)")
scat = scatter!(ax, lat, depth, markersize = 5, color = T, colormap = Reverse(:RdYlBu))
cb = Colorbar(fig[1, 2], scat)
fig

## Extract and plot bathymetry
Its resolution is chosen according to the extension of the region of interest.

In [ ]:
bathname = gebco04file
download_check(gebco04file, gebco04fileURL)

We create a 2D grid for the bathymetry extraction: the goal is to know the bathymetry where we have the profiles, hence the original bathymetry has to be re-interpolated.

In [ ]:
lonmin, lonmax, latmin, latmax = minimum(lon), maximum(lon), minimum(lat), maximum(lat)
Δlon = 0.1#, lonmax - lonmin;
Δlat = latmax - latmin;
xi = lonmin-0.05*Δlon:0.01:lonmax+0.05*Δlon
yi = latmin-0.05*Δlat:0.01:latmax+0.05*Δlat

bx, by, b = DIVAnd.extract_bath(bathname, true, xi, yi)

Plot the bathymetry and the data locations.

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1])
pcm = heatmap!(ax, bx, by, b, colormap = :gist_earth)
scatter!(ax, lon, lat, color = :black, markersize = 4)
Colorbar(fig[1, 2], pcm)
fig

### Create the interpolator

In [ ]:
lon_u, lat_u = unique(lon), unique(lat)
itp = Interpolations.interpolate((bx, by), b, Gridded(Constant()));
depth_interp = itp.(lon_u, lat_u);

Plot the interpolated depth over the bathymetry to see if they correspond.

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1])
pcm = heatmap!(ax, bx, by, b, colormap = :gist_earth, colorrange = [2000.0, 3000.0])
scatter!(
    ax,
    lon_u,
    lat_u,
    color = depth_interp,
    markersize = 4,
    colormap = :gist_earth,
    colorrange = [2000.0, 3000.0],
)
Colorbar(fig[1, 2], pcm)
fig

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], xlabel = "Latitude (°N)", ylabel = "Depth (m)")
scat = scatter!(ax, lat, depth, markersize = 5, color = T, colormap = Reverse(:RdYlBu))
cb = Colorbar(fig[1, 2], scat)
lines!(ax, unique(lat), -depth_interp)
fig

## Create interpolation grid

In [ ]:
Δlatg = 0.02 # degrees North
Δdepth = 1.0 # meter
latgrid = collect(minimum(lat):Δlatg:ceil(maximum(lat)))
depthgrid = collect(0.0:Δdepth:ceil(maximum(-depth)));

### Land sea mask
First we interpolate the bathymetry on the distance grid.

In [ ]:
npoints = length(latgrid)
lon_grid = range(lon[1] - 0.05 * Δlon, lon[end] + 0.05 * Δlon; length = npoints);
lat_grid = range(lat[1], lat[end]; length = npoints)
depth_interp_grid = itp.(lon_grid, lat_grid);

Then we can construct the mask.

In [ ]:
mask = falses(length(lat_grid), length(depthgrid));
for (ii, d) in enumerate(lat_grid)
    for (jj, h) in enumerate(depthgrid)
        if h < depth_interp_grid[ii]
            mask[ii, jj] = true
        end
    end
end;

In [ ]:
heatmap(mask)

## Analysis
### Create metrics

In [ ]:
mask_rect, pmn, xyi = DIVAnd_rectdom(latgrid, depthgrid);

### Set parameters

In [ ]:
len = ( 0.05 .* ones(length(latgrid), length(depthgrid)), 
        5.0 .* ones(length(latgrid), length(depthgrid))) # km, meters
epsilon2 = 5.

### Run analysis
We remove the mean value to get the anomalies.

In [ ]:
Tinterp, T_struct = DIVAndrun(mask_rect, pmn, xyi, (lat, -depth), T .- mean(T), 
    len, epsilon2, alphabc=0);

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], xlabel = "Latitude (°N)", ylabel = "Depth (m)")
heatmap!(ax, latgrid, -depthgrid, Tinterp, colormap = Reverse(:RdYlBu))
xlims!(lat[1], lat[end])
fig